Done:
- Segmentation (1280x960) and Depth (640x480) are aligned
Does
- Align Opical Flow

In [ ]:
import sys
import os
sys.path.append( """/home/jonfrey/ASL/src""")

from torchvision import transforms as tf 
import torch
import time 
from pathlib import Path
import imageio

import matplotlib.pyplot as plt
from time import time
from glob import glob
import matplotlib

import cv2
import numpy as np
from PIL import Image
from scipy import ndimage

# Modules
from visu import Visualizer

In [ ]:
from pseudo_label import *
depth_paths = getPathsDepth()
segmentation_paths = getPathsSegmentation()
flow_paths = getFlowPaths()
depth_paths = getPathsFlow()

In [ ]:
# Images
base = "/home/jonfrey/datasets/scannet"
image_pths = [str(p) for p in glob( base+'/**/*.jpg', recursive=True ) if str(p).find('color') != -1]
fun = lambda x : x.split('/')[-3][-7:] + '_'+ str( "0"*(6-len( x.split('/')[-1][:-4]))) + x.split('/')[-1][:-4]  
image_pths.sort(key=fun)
image_pths

In [ ]:
# Depths
base = "/home/jonfrey/datasets/scannet/scans/scene0000_00/depth_estimate"
depth_pths = [str(p) for p in glob( base+'/**/*.png', recursive=True ) if str(p).find('depth_estimate') != -1 and str(p).find('preview') == -1 ]
fun = lambda x : x.split('/')[-3][-7:] + '_'+ str( "0"*(6-len( x.split('/')[-1][:-4]))) + x.split('/')[-1][:-4]  
depth_pths.sort(key=fun)

depth_pths = [d for d in depth_pths if int( d.split('/')[-1][:-4])%10 == 0]

In [ ]:
# Flows
base = "/home/jonfrey/results/scannet_eval/run_24h_train_1gpu/scannet"
flow_pths = [str(p) for p in glob( base+'/**/*.png', recursive=True ) if str(p).find('flow') != -1]
#flow_low_pths = [str(p) for p in glob( base+'/**/*.png', recursive=True ) if str(p).find('flow_low_') != -1]
fun = lambda x :x.split('/')[-3]+ '0'*(8-len((x.split('/')[-1]).split('_')[-1]))+(x.split('/')[-1]).split('_')[-1]
flow_pths.sort(key=fun)

flow_pths
#flow_low_pths.sort(key=fun)

In [ ]:
# Segmentation
base = "/home/jonfrey/results/scannet_eval/run_24h_train_1gpu"
segmentation_pths = [str(p) for p in glob( base+'/**/*.png', recursive=True ) if str(p).find('segmentation_estimate') != -1]
fun = lambda x : x.split('/')[-3][-7:] + '_'+ str( "0"*(6-len( x.split('/')[-1][:-4]))) + x.split('/')[-1][:-4]  
segmentation_pths.sort(key=fun)

In [ ]:
def readFlowKITTI(filename):
    flow = cv2.imread(filename, cv2.IMREAD_ANYDEPTH|cv2.IMREAD_COLOR)
    flow = flow[:,:,::-1].astype(np.float32)
    flow, valid = flow[:, :, :2], flow[:, :, 2].astype(bool)
    flow = (flow - 2**15) / 64.0
    H,W = 960,1280
    return flow, valid
flow, valid = readFlowKITTI(flow_pths[10])

In [ ]:
def readDepth(filename): 
    H,W = 960,1280
    im = imageio.imread(filename)
    im = im.astype(np.float32)
    im = im / 1000
    im = ndimage.zoom(im, (H/im.shape[0],W/im.shape[1]) , order=1)
    return im
readDepth(depth_pths[0])

In [ ]:
def readSegmentation(filename):
    im = imageio.imread(filename)
    pred = im[:,:,0].astype(np.int32)
    target = im[:,:,1].astype(np.int32)
    valid = im[:,:,2].astype(bool)
    pred -= 1
    target -= 1
    return pred, target, valid
pred, target, valid = readSegmentation(segmentation_pths[300])

In [ ]:
visu = Visualizer(os.getenv('HOME')+'/tmp', logger=None, epoch=0, store=False, num_classes=41)
pred, target, valid = readSegmentation(segmentation_pths[20])
#ret = visu.plot_segmentation(seg=pred, jupyter=True)
ret = visu.plot_segmentation(seg=target, jupyter=True)

In [ ]:
flow, valid = readFlowKITTI(flow_up_pths[20])
ret = visu.plot_flow(flow=flow, jupyter=True)

In [ ]:
depth = readDepth(depth_pths[20])
ret = visu.plot_depth( depth , jupyter=True)